<a href="https://colab.research.google.com/github/PrimWong/light_optics_grating_experiment/blob/main/Prim_Light_Optics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Prim Laser Distance Detection

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
img_dir = '/content/drive/MyDrive/Physic_data/processed_image'

In [3]:
from PIL import Image
import numpy as np

# Open the image file
with Image.open("/content/drive/MyDrive/Physic_data/processed_image/4555_0.jpg") as img:
    # Convert image to grayscale
    gray_img = img.convert("L")

    # Convert image data to a numpy array
    image_array = np.array(gray_img)

    # Since the points are white and the background is not, we can use a high threshold to find the white points
    # Threshold value close to 255 (white) to identify the white points
    threshold = 250
    white_points = np.where(image_array > threshold)

    # The `np.where` function returns a tuple of arrays, one for each dimension of the array,
    # where the truth value of the array is non-zero
    # Here we are interested in the x-coordinates (i.e., the second array) because we want to measure the horizontal distance
    x_coords = white_points[1]

    # Sort the x-coordinates to ensure we have them in order from left to right
    sorted_x_coords = np.sort(x_coords)

# We are interested only in the first occurrence of each white point since they are horizontally aligned
# Thus, we'll find the unique set of x-coordinates, assuming a point's width does not exceed a certain number of pixels
# We'll assume a max width of 5 pixels for a white point
unique_x_coords = sorted_x_coords[np.unique(sorted_x_coords, return_index=True)[1]]

# Calculate the distances between the consecutive white points
distances = np.diff(unique_x_coords)

distances


array([6, 5, 1, ..., 1, 1, 1])

In [4]:
import cv2

# Load the image
image = cv2.imread("/content/drive/MyDrive/Physic_data/processed_image/4555_0.jpg")

# Convert the image to grayscale
gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# Threshold the image to get the white points (dots)
_, thresholded = cv2.threshold(gray, 250, 255, cv2.THRESH_BINARY)

# Find contours of the white points
contours, _ = cv2.findContours(thresholded, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Calculate the center of the contours
centers = []
for contour in contours:
    M = cv2.moments(contour)
    # To avoid division by zero check if the area (M["m00"]) is not zero
    if M["m00"] != 0:
        center_x = int(M["m10"] / M["m00"])
        center_y = int(M["m01"] / M["m00"])
        centers.append((center_x, center_y))

# Sort the center points based on x coordinate
centers_sorted = sorted(centers, key=lambda point: point[0])

# Assuming the contours of interest are the three with the largest area
# This is because the white points are the most significant features on the graph line
largest_contours = sorted(contours, key=cv2.contourArea, reverse=True)[:3]

# Calculate the center of the largest contours
largest_centers = []
for contour in largest_contours:
    M = cv2.moments(contour)
    if M["m00"] != 0:
        center_x = int(M["m10"] / M["m00"])
        center_y = int(M["m01"] / M["m00"])
        largest_centers.append((center_x, center_y))

# Sort these centers based on x coordinate as well
largest_centers_sorted = sorted(largest_centers, key=lambda point: point[0])

# Calculate the distances between the consecutive white points
if len(largest_centers_sorted) >= 2:
    distance_1 = largest_centers_sorted[1][0] - largest_centers_sorted[0][0]
    distance_2 = largest_centers_sorted[2][0] - largest_centers_sorted[1][0]
    distances_between_points = (distance_1, distance_2)
else:
    distances_between_points = ()

distances_between_points


(555, 568)

In [5]:
# First, we need to redefine 'w' as the width of the image. This was lost due to the stateless nature of the interaction.
# Re-open the image to get its width
with Image.open("/content/drive/MyDrive/Physic_data/processed_image/4555_0.jpg") as img:
    w, h = img.size  # width and height of the image

# The user has now provided the complete scale which is from 12.2 to 0 and then from 0 to 10.7.
# The total number of units across the width of the image is therefore 12.2 + 10.7.
total_scale_units_corrected = 12.2 + 10.7

# The distance between the two '0' points is equal to the width of the image
distance_between_zeros_px_corrected = w

# Calculate the pixel distance of one scale unit with the updated total scale units
scale_unit_px_corrected = distance_between_zeros_px_corrected / total_scale_units_corrected

# Convert the pixel distances between white points into scale units with the corrected scale
distances_in_scale_units_corrected = [d / scale_unit_px_corrected for d in distances_between_points]

scale_unit_px_corrected, distances_in_scale_units_corrected


(130.13100436681225, [4.264932885906039, 4.3648322147651])

In [14]:
# First, we need to redefine 'w' as the width of the image. This was lost due to the stateless nature of the interaction.
# Re-open the image to get its width
with Image.open("/content/drive/MyDrive/Physic_data/processed_image/4555_0.jpg") as img:
    w, h = img.size  # width and height of the image

# The user has now provided the complete scale which is from 12.2 to 0 and then from 0 to 10.7.
# The total number of units across the width of the image is therefore 12.2 + 10.7.
total_scale_units_corrected = 12.2 + 10.7

# The distance between the two '0' points is equal to the width of the image
distance_between_zeros_px_corrected = w

# Calculate the pixel distance of one scale unit with the updated total scale units
scale_unit_px_corrected = distance_between_zeros_px_corrected / total_scale_units_corrected

# Convert the pixel distances between white points into scale units with the corrected scale
distances_in_scale_units_corrected = [d / scale_unit_px_corrected for d in distances_between_points]

distances_in_scale_units_corrected


[4.264932885906039, 4.3648322147651]

In [13]:
import math

# Given L
L = 13

# Distances in units from the previous calculation
first_distance_units = distances_in_scale_units_corrected[0]
second_distance_units = distances_in_scale_units_corrected[1]

# Calculate the angles using the inverse tangent function
theta_1 = math.atan(first_distance_units / L)
theta_2 = math.atan(second_distance_units / L)

# Convert the angles from radians to degrees for a more common understanding
theta_1_degrees = math.degrees(theta_1)
theta_2_degrees = math.degrees(theta_2)

# Calculate theta_average
theta_average_degrees = (theta_1_degrees + theta_2_degrees) / 2

# Convert theta_average to radians for calculation of sine
theta_average_radians = math.radians(theta_average_degrees)

# Calculate sin(theta_average)
sin_theta_average = math.sin(theta_average_radians)

# Given value of d
d = 1.8955e-6  # in meters

# Calculate lambda using the formula lambda = d * sin(theta_average)
lambda_value = d * sin_theta_average

theta_average_degrees, sin_theta_average, lambda_value
# Convert lambda value from meters to nanometers
lambda_value_nm = lambda_value * 1e9  # 1 meter = 1e9 nanometers

lambda_value_nm

597.1041226122119

In [19]:
import math

def calculate_wavelength(L, N):
    # Convert N from lines per inch to lines per meter (1 inch = 0.0254 meters)
    N_per_meter = N / 0.0254

    # Calculate d as the reciprocal of N (in meters)
    d = 1 / N_per_meter

    # Distances in units from previous calculations
    first_distance_units = distances_in_scale_units_corrected[0]
    second_distance_units = distances_in_scale_units_corrected[1]

    # Calculate the angles using the inverse tangent function
    theta_1 = math.atan(first_distance_units / L)
    theta_2 = math.atan(second_distance_units / L)

    # Convert the angles from radians to degrees
    theta_1_degrees = math.degrees(theta_1)
    theta_2_degrees = math.degrees(theta_2)

    # Calculate theta_average in degrees
    theta_average_degrees = (theta_1_degrees + theta_2_degrees) / 2

    # Convert theta_average to radians for calculation of sine
    theta_average_radians = math.radians(theta_average_degrees)

    # Calculate sin(theta_average)
    sin_theta_average = math.sin(theta_average_radians)

    # Calculate lambda using the formula lambda = d * sin(theta_average)
    lambda_value = d * sin_theta_average

    # Convert lambda value from meters to nanometers (1 meter = 1e9 nanometers)
    lambda_value_nm = lambda_value * 1e9

    return theta_average_degrees, sin_theta_average, lambda_value_nm

L = float(input("Enter the value for L (distance in graph units, suggested value = 13): "))
N = int(input("Enter the value for N (number of lines per inch in the grating, suggested value = 13400): "))

theta_avg, sin_theta_avg, lambda_nm = calculate_wavelength(L,N)
print(f"Theta Average (Degrees): {theta_avg}")
print(f"Sin(Theta Average): {sin_theta_avg}")
print(f"Lambda (nm): {lambda_nm:.2f} nm")

Enter the value for L (distance in graph units, suggested value = 13): 13
Enter the value for N (number of lines per inch in the grating, suggested value = 13400): 13400
Theta Average (Degrees): 18.361502445758095
Sin(Theta Average): 0.31501140733960004
Lambda (nm): 597.11 nm
